# DeepSeek R1 14B Colab Inference

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TimS-ml/nanoGPT-mod/blob/master/misc/Deepseek_r1_14b_colab.ipynb)

[![Open In GitHub](https://img.shields.io/badge/Open%20In%20GitHub-black?style=flat&logo=github)](https://github.com/TimS-ml/nanoGPT-mod/blob/master/misc/Deepseek_r1_14b_colab.ipynb)

In [ ]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

In [6]:
!pip install -q unsloth
!pip uninstall -q unsloth -y && pip install -q --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from unsloth import FastLanguageModel
import torch

# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit"
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-14B-unsloth-bnb-4bit"

max_seq_length = 2048
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
)

In [9]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120, padding_idx=151654)
    (layers): ModuleList(
      (0-4): 5 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
      )
 

In [10]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 1.5, min_p = 0.1
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<think>
The sequence starts with 1, 1, and then each subsequent number is the sum of the two preceding ones. After 5, the next number should be 8, which already exists in the provided sequence. Adding the next term involves adding the two previous numbers: 5 + 8 equals 13.

Thus, the extended sequence includes the number 13 following 8.
</think>

Sure! Let's continue the Fibonacci sequence from where it left off:

**Given sequence:**  
1, 1, 2, 3, 5, 8,  
**Next terms:**

The Fibonacci


In [13]:
messages = [
    {"role": "user", "content": "Once upon a time, there was a"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True, temperature = 1.5, min_p = 0.1
)

<think>
Okay, so I need to figure out how to respond to this. The user wrote a story starter: "Once upon a time, there was a..." and they provided a response from someone called "the story whisperer" who continued the story. The response included a story about a small coastal village, the old lighthouse keeper named Thomas, his dog Max, a mysterious fog, and a little girl named Clara who could communicate with the sea. The user then wrote their own version, adding details like the wooden lighthouse with a rusty iron staircase, the creaking floorboards, the salty tang, and the foghorn
